Оценка качества модели для каждой комбинации гиперпараметров является дорогостоящей частью оптимизации — на приведённых крохотных сетках по нескольким параметрам это занимает минуты, но в реальных масштабах это занимает часы и сутки. Поэтому в идеале мы хотим оптимизировать гиперпараметры самым эффективным образом. 

Один из способов  — это байесовская оптимизация. Она отличается от случайного поиска или поиска по сетке тем, что учитывает предыдущие результаты, а не выбирает комбинации из вариантов, не имеющих информации о прошлых оценках. Во многих случаях это позволяет найти лучшие значения гиперпараметров модели за меньшее количество времени. Таким образом, мы получаем и более быструю оптимизацию, и более качественный результат. Это два желаемых результата, особенно когда мы работаем с настройкой гиперпараметров моделей МО.

Существует несколько разных алгоритмов для этого типа оптимизации, но особенно используемым является Tree-Structured Parzen Estimators (TPE).

Tree-Structured Parzen Estimators (TPE)

1На каждой итерации алгоритм TPE учитывает информацию о прошлых опробованных комбинациях гиперпараметров и только потом принимает решение, какой набор следует попробовать дальше. 

Чтобы приступить к использованию TPE, необходимо выполнить несколько итераций с помощью случайного поиска. 

2На следующем шаге происходит разделение собранных наборов на две группы:

в первую группу входят наборы, дающие наилучшие результаты после оценки;
во вторую — все остальные.
На изображении ниже: первая группа — красные точки находятся в области минимума целевой функции; вторая группа — синие точки, все остальные.



Источник

Основная цель алгоритма — найти набор гиперпараметров, который с большей вероятностью будет в первой группе и с меньшей вероятностью во второй группе. Таким образом, для принятия следующего решения используется целое распределение наилучших комбинаций — красные точки на графике.

3Далее TPE моделирует вероятности правдоподобия для каждой из групп, используя формулу Байеса:

где  — гиперпараметры,  — соответствующая оценка качества модели.

Подробнее про теорему Байеса можно прочитать по ссылке.

4Затем, используя вероятность правдоподобия из первой группы, отбирается набор комбинаций, которые с большей вероятностью попадут в первую группу и с меньшей вероятностью — во вторую. 

где  — это вероятность гиперпараметров с учётом оценки целевой функции,  — вероятность быть в первой группе,  — вероятность быть во второй группе

Определяем ожидаемое улучшение для каждой комбинации:

где  — это знак математического ожидания.
5Шаги 2-4  будет выполняться до тех пор, пока не будет достигнуто максимальное количество итераций. 

В итоге мы найдём наилучшую комбинацию гиперпараметров.

Более подробное описание алгоритма с математическими выкладками вы можете найти здесь.

Hyperopt

Hyperopt — это библиотека Python с открытым исходным кодом на основе байесовской оптимизации, в которой реализован алгоритм Tree-Structured Parzen Estimators (TPE).

Три шага для использования Hyperopt:

1Задание пространства поиска гиперпараметров. 

Объявляем список гиперпараметров, тип распределения и его границы.

Основные (наиболее часто используемые) типы:

hp.choice(label, options) — равновероятный выбор из массива. Массив (список/кортеж) вы задаёте сами, в списке могут быть как числа, так и строки (категории), но, как правило, данный метод используется для оптимизации категориального гиперпараметра (например, тип регуляризации в линейной регрессии или критерий информативности в деревьях);
hp.randint(label, upper) — возвращает случайное целое число из диапазона [0, upper];
hp.uniform(label, low, high) — создаёт равномерное непрерывное распределение и возвращает случайное число (не обязательно целое) из диапазона [low, high];
hp.normal(label, mu, sigma) — создаёт нормальное непрерывное распределение с параметрами mu и sigma и возвращает случайное число из этого распределения;
hp.lognormal(label, mu, sigma) — создаёт логнормальное непрерывное распределение с параметрами mu и sigma и возвращает случайное число из этого распределения.
2Задание целевой функции. 

Создаём модель МО, передаём ей данные и оцениваем её на основе выбранной метрики. Можем минимизировать/максимизировать значение метрики.

3Задание алгоритма поиска:

Random Search.
Tree of Parzen Estimators (TPE).
Полезные ссылки: 

Hyperopt: A Python Library for Optimizing the Hyperparameters of Machine Learning Algorithms
Hyperopt на GitHub
Байесовский ниндзя (Хабр)

Плюсы и минусы рассмотренных фреймворков и библиотек

Название	Плюсы 	Минусы
GridSearchCV	
✔️ Простой в использовании, идеален для небольшой сетки гиперпараметров

✔️ Встроенная кросс-валидация

✔️ Включает построение итоговой модели на всей обучающей выборке 

⛔️ Перебирает просто все комбинации заданной сетки гиперпараметров 

⛔️  Требует много времени и вычислительных ресурсов

RandomizedSearchCV	✔️ Эффективнее и экономичнее gridsearchcv
✔️ Можно задать количество рассматриваемых комбинаций

⛔️ Просто выбирает рандомные комбинации гиперпараметров без учёта результатов прошлых итераций
Hyperopt	✔️ Быстрее и эффективнее, чем классические методы перебора (GSCV и RSCV)
✔️ Учитывает результаты прошлых итераций — байесовский оптимизатор

✔️ Можно отслеживать дополнительную информацию на каждом шаге с помощью класса Trials

✔️ Возможность построения условного пространства поиска гиперпараметров и даже моделей

⛔️ Старая документация, плохо с поддержкой и обновлением 
⛔️ Способен только минимизировать

⛔️ Непростой синтаксис описания пространства гиперпараметров (особенно с использованием условных реализаций)

Optuna	✔️ Один из самых быстрых и эффективных
✔️ Специально разработан для оптимизации гиперпараметров 

✔️ Простой в использовании

✔️ Относительно новый фреймворк с хорошей документацией 

✔️ Учитывает результаты прошлых итераций — байесовский оптимизатор 

✔️ Встроенная визуализация результатов

✔️ Возможность явно задавать максимизацию или минимизацию функции качества

⛔️ Удаление «плохих» точек пространства из рассмотрения
⛔️ Не стоит полностью полагаться: важно с умом определять пространство поиска, что может значительно сократить время расчётов.